In [2]:
import pandas as pd
import numpy as np
import re  # string mani
import os  # read fold info

In [280]:
# functions:
def d_overview(df):
    print(f"Dimension: {df.shape}")
    output = df.apply(lambda x: (x.dtype,x.isna().mean(),x.unique()), axis=0).T
    output.columns = ["dtype", "NaN_precentage","unique"]
    return output
def rm_14(df, check=False):
    if check:
        print(df.apply(lambda x:sum(x.isin(["-1","-4",-1,-4]))))
    df.replace([-1,-4,"-1","-4"], np.nan, inplace=True)
def keep_median_row(group):
    if 'MEDIAN' in group['BATCH'].values:
        return group[group['BATCH'] == 'MEDIAN']
    else:
        return group.head(1)
def basic_clean(df):
    df = df.loc[:, (df.isna().mean() <= 0.5) &   # rm >50% missingness
                ~(df.columns.str.contains('DATE'))  # drop dates
               ].reset_index(drop=True) 
    df = df.drop('update_stamp', axis=1)
    return df

In [281]:
path = "../../ADNI_testing/"
biomarker = pd.read_csv(f"{path}UPENNBIOMK_MASTER.csv")     # adni1
diagnosis = pd.read_csv(f"{path}DXSUM_PDXCONV_ADNIALL.csv")
symptoms = pd.read_csv(f"{path}ADSXLIST.csv")
bl_symptoms = pd.read_csv(f"{path}BLSCHECK.csv")

In [282]:
# turn -1,-4 to NaN
rm_14(biomarker) ; rm_14(diagnosis) ; rm_14(symptoms) ; rm_14(bl_symptoms)

## biomarker
# keep the median value or single row batch:
biomarker = biomarker.groupby(['RID', 'VISCODE'], as_index=False) \
.apply(keep_median_row).reset_index(drop=True)
# keep useful cols:
biomarker = biomarker.loc[:,['RID','VISCODE','ABETA','TAU','PTAU']]

## diagnosis:
diagnosis=diagnosis.loc[diagnosis["Phase"]!="ADNI3"] # rm ADNI3
diagnosis = basic_clean(diagnosis)

## symptoms:
symptoms = basic_clean(symptoms)
## bl_symptoms
bl_symptoms = basic_clean(bl_symptoms)

## merge the 4 dataset:

array(['bl', 'm12', 'm24', 'm48', 'm60', 'm84', 'm36', 'm72'],
      dtype=object)

In [252]:
# d_overview(biomarker)

diagnosis.loc[(diagnosis.VISCODE != diagnosis.VISCODE2),:]

Phase       ID   RID        PTID  SITEID VISCODE VISCODE2    USERDATE  \
4343   ADNI2     10.0  4007  072_S_4007      34     v01       sc  2011-03-23   
4344   ADNI2     12.0  4003  098_S_4003      38     v01       sc  2011-03-23   
4345   ADNI2     14.0   172  098_S_0172      38     v06      m60  2011-03-23   
4346   ADNI2     16.0  4002  098_S_4002      38     v01       sc  2011-03-24   
4347   ADNI2     20.0  4010  116_S_4010      43     v01       sc  2011-03-28   
...      ...      ...   ...         ...     ...     ...      ...         ...   
10009  ADNI2  11386.0   919  053_S_0919      30     v41     m120  2019-03-22   
10010  ADNI2  11388.0  2396  053_S_2396      30     v41      m60  2019-04-03   
10011  ADNI2  11390.0  4557  053_S_4557      30     v51      m60  2019-04-03   
10012  ADNI2  11392.0  1256  082_S_1256      36     v31      m84  2019-04-12   
10013  ADNI2  11394.0  2391  130_S_2391      50     v41      m60  2019-04-15   

         EXAMDATE  DXCHANGE  DXPARK  DXDEP           update_stamp  
4343   2011-03-04       2.0     0.0    0.0  2013-03-14 19:16:45.0  
4344   2011-02-22       1.0     0.0    0.0  2013-03-14 19:16:45.0  
4345   2011-03-14       1.0     0.0    0.0  2013-03-14 19:16:45.0  
4346   2011-02-14       1.0     0.0    0.0  2013-03-14 19:16:45.0  
4347   2011-03-28       1.0     0.0    0.0  2013-03-14 19:16:45.0  
...           ...       ...     ...    ...                    ...  
10009  2016-11-10       2.0     1.0    0.0  2019-03-22 19:48:09.0  
10010  2016-09-09       2.0     0.0    0.0  2019-04-03 19:46:02.0  
10011  2017-03-22       2.0     0.0    0.0  2019-04-03 19:46:02.0  
10012  2014-05-12       1.0     0.0    0.0  2019-04-12 19:46:06.0  
10013  2019-04-15       3.0     0.0    0.0  2019-04-15 19:46:03.0  

[5671 rows x 13 columns]

In [145]:
pd.merge(df1, df2, on="foreign_key", how=["left", "inner", "right"])

Phase            object
ID              float64
RID               int64
PTID             object
SITEID            int64
VISCODE          object
VISCODE2         object
USERDATE         object
USERDATE2        object
EXAMDATE         object
DXCHANGE        float64
DXCURREN        float64
DXCONV          float64
DXCONTYP        float64
DXREV           float64
DXNORM          float64
DXNODEP         float64
DXMCI           float64
DXMDES           object
DXMPTR1         float64
DXMPTR2         float64
DXMPTR3         float64
DXMPTR4         float64
DXMPTR5         float64
DXMPTR6         float64
DXMDUE          float64
DXMOTHET         object
DXMOTHSP         object
DXDSEV          float64
DXDDUE          float64
DXAD            float64
DXADES          float64
DXAPP           float64
DXAPROB          object
DXAMETASP        object
DXAOTHRSP        object
DXAPOSS          object
DXAATYSP         object
DXAMETSP         object
DXAOTHSP         object
DXPARK          float64
DXPARKSP        

In [213]:
# biomarker:
d_overview(biomarker)


Dimension: (2174, 14)


dtype NaN_precentage  \
RID             int64            0.0   
VISCODE        object            0.0   
BATCH          object            0.0   
KIT            object            0.0   
STDS           object            0.0   
DRAWDTE        object            0.0   
RUNDATE        object            0.0   
ABETA         float64        0.00092   
TAU           float64        0.01242   
PTAU          float64        0.00138   
ABETA_RAW     float64            1.0   
TAU_RAW       float64            1.0   
PTAU_RAW      float64            1.0   
update_stamp   object            0.0   

                                                         unique  
RID           [3, 4, 5, 8, 10, 14, 16, 19, 22, 23, 29, 31, 3...  
VISCODE                 [bl, m12, m24, m48, m60, m84, m36, m72]  
BATCH                                                  [MEDIAN]  
KIT                                                       [ALL]  
STDS                                                      [ALL]  
DRAWDTE       [2005-09-12, 2006-09-13, 2005-11-22, 2006-11-2...  
RUNDATE                                            [2016-03-09]  
ABETA         [131.0, 137.0, 256.0, 246.0, 116.0, 136.0, 268...  
TAU           [61.4, 76.5, 41.8, 49.4, 116.0, 120.0, 125.0, ...  
PTAU          [20.4, 21.1, 12.9, 13.5, 67.2, 40.5, 29.0, 39....  
ABETA_RAW                                                 [nan]  
TAU_RAW                                                   [nan]  
PTAU_RAW                                                  [nan]  
update_stamp  [2016-07-06 16:15:51.0, 2016-07-06 16:15:52.0,...

method

In [54]:
print(len(biomarker.RID.unique()))
biomarker.loc[(biomarker["VISCODE"]=="bl") &
             (biomarker["BATCH"]=="MEDIAN")
             ].RID.shape
biomarker.BATCH.value_counts()

print(pd.crosstab(biomarker["VISCODE"], biomarker["BATCH"]))
# if duplicate -> keep median, if not -> keep 

# group by RID -> keep duplicate or median
# biomarker.drop_duplicates(subset="RID")


<bound method GroupBy.all of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f82b4d98590>>

In [83]:
type(biomarker)

pandas.core.frame.DataFrame

In [28]:
# merge biomarker files:
fls = os.listdir(path)
fl_biomarkers = [x for x in fls if ("UPENNBIOMK" in x) and ("DICT" not in x)]
# UPENNBIOMK12_01_04_21 = pd.read_csv(f"{path}{fl_biomarkers[0]}")
# UPENNBIOMK10_07_29_19 = pd.read_csv(f"{path}{fl_biomarkers[1]}")
UPENNBIOMKADNIDIAN2017 = pd.read_csv(f"{path}{fl_biomarkers[2]}")
# UPENNBIOMK9_04_19_17 = pd.read_csv(f"{path}{fl_biomarkers[3]}")
# biomarker = pd.read_csv(f"{path}{fl_biomarkers[-1]}")


for nms in fl_biomarkers:
    to_concat = pd.read_csv(f"{path}{nms}")
    print(to_concat.columns)
    # biomarker = pd.concat([biomarker, to_concat])

Index(['RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'PROT', 'RUNDATE', 'AB40',
       'ABETA', 'PTAU', 'TAU', 'AB4240', 'NOTE'],
      dtype='object')
Index(['RID', 'VISCODE', 'VISCODE2', 'DRAWDATE', 'RUNDATE', 'ABETA40',
       'ABETA42', 'PTAU', 'TAU', 'NOTE', 'update_stamp'],
      dtype='object')
Index(['RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'STUDY', 'RUNDATE', 'ABETA',
       'AB40', 'TAU', 'PTAU', 'A4240', 'NOTE', 'update_stamp'],
      dtype='object')
Index(['RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'PHASE', 'BATCH', 'KIT',
       'STDS', 'RUNDATE', 'ABETA', 'TAU', 'PTAU', 'COMMENT', 'update_stamp'],
      dtype='object')
Index(['RID', 'VISCODE', 'BATCH', 'KIT', 'STDS', 'DRAWDTE', 'RUNDATE', 'ABETA',
       'TAU', 'PTAU', 'ABETA_RAW', 'TAU_RAW', 'PTAU_RAW', 'update_stamp'],
      dtype='object')


In [37]:
to_concat.columns

Index(['RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'PHASE', 'BATCH', 'KIT',
       'STDS', 'RUNDATE', 'ABETA', 'TAU', 'PTAU', 'COMMENT', 'update_stamp'],
      dtype='object')

In [34]:
# testd = biomarker.tail(1000)

In [ ]:
testd.join(diagnosis, on=[])